In [1]:
import os

os.environ["SPARK_HOME"] = "/Users/admin/Personal/Code/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"

In [2]:
%pip install pyspark
%pip install 'pyspark[sql]'
%pip install 'pyspark[connect]'
%pip install matplotlib
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [3]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [62]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.pyspark.python", "/usr/bin/python3")
conf.set("spark.pyspark.driver.python", "/usr/bin/python3")
conf.set("spark.driver.memory", "6g")
conf.set("spark.executor.memory", "6g")

spark = SparkSession.builder \
    .master("local[4]") \
    .config(conf=conf) \
    .appName("Lab3") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark

In [73]:
import pandas as pd

# 1. Завантажимо дані
df = pd.read_csv("/Users/admin/Personal/KPI/Big Data/lab3/japanese_universities.csv", index_col=None)

In [81]:
# Заповнимо клітинки зі значенням NaN нулями
df = df.fillna(0)

In [82]:
# 2. Створимо Spark DataFrame
sparkDf = spark.createDataFrame(df)
sparkDf.show(1)

+---+-------------+-------------------+----------+--------+-------+----------------------------+-----------+--------------+---------+--------+---------+----------+-------+-------------+----------------+--------+----------+-------------+------------+-------------+---------------+
| id|         code|               name|   name_jp|    type|type_jp|                     address|postal_code|         phone|    state|state_jp| latitude| longitude|  found|faculty_count|department_count|has_grad|has_remote|review_rating|review_count|difficulty_SD|difficulty_rank|
+---+-------------+-------------------+----------+--------+-------+----------------------------+-----------+--------------+---------+--------+---------+----------+-------+-------------+----------------+--------+----------+-------------+------------+-------------+---------------+
|  0|F101110100010|Hokkaido University|北海道大学|National|   国立|北海道札幌市北区北8条西5丁目|  060－0808|011－716－2111|Hokkai Do|  北海道|43.070446|141.347153|1876-08|           33| 

/Users/admin/Library/Python/3.9/lib/python/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [83]:
# 3. Виведемо count з групуванням за полем state
sparkDf.groupBy('state').count().show()

+-------------+-----+
|        state|count|
+-------------+-----+
| Nagasaki Ken|    8|
|    Shiga Ken|    9|
|   Kagawa Ken|    4|
|   Aomori Ken|   10|
|  Niigata Ken|   22|
|     Tokyo To|  146|
|    Kochi Ken|    5|
|    Gunma Ken|   15|
|  Okinawa Ken|    8|
|Hiroshima Ken|   21|
|  Tochigi Ken|    9|
| Kumamoto Ken|    9|
|     Oita Ken|    5|
|     Nara Ken|   11|
|    Hokkai Do|   37|
|    Hyogo Ken|   36|
| Ishikawa Ken|   14|
|   Nagano Ken|   11|
|   Miyagi Ken|   14|
|    Iwate Ken|    6|
+-------------+-----+


In [84]:
# 4. Створимо датасет з наявного, в якому рейтинги > 3.8 (поле review_rating)
highly_rated_df = sparkDf.where(sparkDf['review_rating'] > 3.8)
highly_rated_df.show(10)

+---+-------------+--------------------+------------+--------+-------+-------------------------------+-----------+--------------+-----------+--------+---------+----------+-------+-------------+----------------+--------+----------+-------------+------------+-------------+---------------+
| id|         code|                name|     name_jp|    type|type_jp|                        address|postal_code|         phone|      state|state_jp| latitude| longitude|  found|faculty_count|department_count|has_grad|has_remote|review_rating|review_count|difficulty_SD|difficulty_rank|
+---+-------------+--------------------+------------+--------+-------+-------------------------------+-----------+--------------+-----------+--------+---------+----------+-------+-------------+----------------+--------+----------+-------------+------------+-------------+---------------+
|  0|F101110100010| Hokkaido University|  北海道大学|National|   国立|   北海道札幌市北区北8条西5丁目|  060－0808|011－716－2111|  Hokkai Do|  北海道|43.070446|14

In [88]:
# 5. Створимо pipeline для кластеризації з такими особливостями:
# а. Виконаємо обробку категоріальних даних / one hot encoding

from pyspark.ml.feature import StringIndexer, OneHotEncoder, MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline

category_columns = ["type", "state", "difficulty_rank"]
numeric_columns = ["latitude", "longitude", "faculty_count", "review_rating"]

# Ми будемо проводити нормалізацію по різному для цих колонок
feature_df = sparkDf.select('id', *category_columns, *numeric_columns)

stages = []

category_index_columns = [col + '_index' for col in category_columns]
category_encoded_columns = [col + '_encoded' for col in category_columns]

string_indexer = StringIndexer(inputCols=category_columns, outputCols=category_index_columns)
onehot_encoder = OneHotEncoder(inputCols=category_index_columns, outputCols=category_encoded_columns)

scaler_vector = VectorAssembler(inputCols=numeric_columns, outputCol='numeric_vector')
scaler = MinMaxScaler(inputCol='numeric_vector', outputCol='numeric_vector_scaled')

stages = [string_indexer, onehot_encoder, scaler_vector, scaler]

pipeline = Pipeline(stages=stages)

In [89]:
model = pipeline.fit(feature_df)
transformed = model.transform(feature_df)
transformed.show(1)

+---+--------+---------+---------------+---------+----------+-------------+-------------+----------+-----------+---------------------+------------+--------------+-----------------------+--------------------+---------------------+
| id|    type|    state|difficulty_rank| latitude| longitude|faculty_count|review_rating|type_index|state_index|difficulty_rank_index|type_encoded| state_encoded|difficulty_rank_encoded|      numeric_vector|numeric_vector_scaled|
+---+--------+---------+---------------+---------+----------+-------------+-------------+----------+-----------+---------------------+------------+--------------+-----------------------+--------------------+---------------------+
|  0|National|Hokkai Do|              A|43.070446|141.347153|           33|         4.16|       2.0|        3.0|                  6.0|   (2,[],[])|(46,[3],[1.0])|          (7,[6],[1.0])|[43.070446,141.34...| [0.87941900058040...|
+---+--------+---------+---------------+---------+----------+-------------+-----

In [104]:
# Застосуємо метод KMeans
from pyspark.ml.clustering import KMeans

features_vector = VectorAssembler(inputCols=['numeric_vector_scaled', *category_encoded_columns], outputCol='features')
kmeans = KMeans(k=3, featuresCol='features')

clustering_pipeline = Pipeline(stages=[features_vector, kmeans])

kmeans_model = clustering_pipeline.fit(transformed)
clustered_df = kmeans_model.transform(transformed)
clustered_df.show(1)

+---+--------+---------+---------------+---------+----------+-------------+-------------+----------+-----------+---------------------+------------+--------------+-----------------------+--------------------+---------------------+--------------------+----------+
| id|    type|    state|difficulty_rank| latitude| longitude|faculty_count|review_rating|type_index|state_index|difficulty_rank_index|type_encoded| state_encoded|difficulty_rank_encoded|      numeric_vector|numeric_vector_scaled|            features|prediction|
+---+--------+---------+---------------+---------+----------+-------------+-------------+----------+-----------+---------------------+------------+--------------+-----------------------+--------------------+---------------------+--------------------+----------+
|  0|National|Hokkai Do|              A|43.070446|141.347153|           33|         4.16|       2.0|        3.0|                  6.0|   (2,[],[])|(46,[3],[1.0])|          (7,[6],[1.0])|[43.070446,141.34...| [0.879

In [106]:
# Згрупуємо дані по кластерам
clustered_df.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  320|
|         2|  144|
|         0|  349|
+----------+-----+


In [112]:
# Застосуємо метод BisectingKMeans
from pyspark.ml.clustering import BisectingKMeans

bisecting_kmeans = BisectingKMeans(k=3, featuresCol='features')
clustering_pipeline = Pipeline(stages=[features_vector, bisecting_kmeans])
bisecting_kmeans_model = clustering_pipeline.fit(transformed)
clustered_df = kmeans_model.transform(transformed)
clustered_df.show(1)

+---+--------+---------+---------------+---------+----------+-------------+-------------+----------+-----------+---------------------+------------+--------------+-----------------------+--------------------+---------------------+--------------------+----------+
| id|    type|    state|difficulty_rank| latitude| longitude|faculty_count|review_rating|type_index|state_index|difficulty_rank_index|type_encoded| state_encoded|difficulty_rank_encoded|      numeric_vector|numeric_vector_scaled|            features|prediction|
+---+--------+---------+---------------+---------+----------+-------------+-------------+----------+-----------+---------------------+------------+--------------+-----------------------+--------------------+---------------------+--------------------+----------+
|  0|National|Hokkai Do|              A|43.070446|141.347153|           33|         4.16|       2.0|        3.0|                  6.0|   (2,[],[])|(46,[3],[1.0])|          (7,[6],[1.0])|[43.070446,141.34...| [0.879

In [113]:
# Згрупуємо дані по кластерам
clustered_df.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  320|
|         2|  144|
|         0|  349|
+----------+-----+


In [8]:
spark.stop()